# Modeleringsprosjekt 2020

# Oppagve 5: Numerisk integrasjon

Integrasjon er en metode i mattematikk som brukes for å finne arealet under en graf.  I analytisk matte bruker man anti-derivasjon til å finne dette arealet, men i numerisk matte er det flere ulike metoder.  I denne Notebooken holder vi oss til bestemte integraler.  Dette betyr at man integrerer en funksjon i et interval.  Da finner man arealet under grafen til funksjonen i dette intervallet.  Under skal vi se ulike metoder for å gjøre dette.

## Rektangelmetoden

### Teori

Rektangelmetoden, også kalt Riemann sum, går ut på at man setter mange rektangler under grafen.  Disse har liten bredde og en høyde tilsvarende funksjonsverdien i et punkt.  Legger man sammen arealet av disse rektanglene får man noe som tilsvarer arealet under grafen.  Dersom man lar antall rektangler gå mot uendelig vil arealet av dem nærme seg arealet under grafen.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/LeftRiemann2.svg/1280px-LeftRiemann2.svg.png" alt="Riemann sum illustrasjon" style="width: 200px; float: left"/>
Figuren er en illustrasjon av at rektangler under grafen kan tilnærme arealet under grafen.

Det mattematiske uttrykket for dette vil være slik:

$$\int_{a}^{b} {f(x) dx} \approx \lim_{N \rightarrow \infty} (\Delta x \sum_{i = 0}^{N-1} {f(a + i * \Delta x)})$$

Der:

$$\Delta x = \frac{b - a}{N}$$

I disse uttrykkene representerer $N$ antall rektangler, $a$ er starten på det bestemte integralet, $b$ er slutten på det bestemte integralet og $\Delta x$ er bredden på rektangelene.

I summen over har vi faktoriser ut $\Delta x$, siden dette er en faktor i alle leddene.  Dette vil føre til et mer effektivt program.  Arealet av et rektangel er gitt ved $A = l * b$.  Her er bredden $\Delta x$ og lengden er høyden til grafen i et punkt, altså $f(a + i * \Delta x)$

### Kode

In [1]:
def RektangelMetoden(f, a, b, N = 100000):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall rektangler
    
    # Bredden per rektangel
    dx = (b - a) / N
    
    # Arealet under grafen
    A = 0
    
    # Loop gjennom alle rekangelene
    for i in range(N):
        # Legg sammen høyden i rektanglene
        A += f(a + i * dx)
    
    # Multipliser med bredden per integral for å finne arealet
    A *= dx

    # returner arealet under grafen f
    return A

Dette kan vi teste med en enkel funksjon som $f(x) = x^2$

In [2]:
def f(x):
    return x**2

Areal = RektangelMetoden(f, 1, 2)
print(Areal)

2.333318333350027


Dette svaret er nærme det analytiske svaret som er på $\frac{7}{3} \approx 2.3333333333333$

### Ulikt antall rektangler
Hvor nærme vi kommer avgjøres av funksjonen og hvor mange rektangler vi har.  Under prøver vi andre funksjoner med ulikt antall rektangler.

In [3]:
def g(x):
    return 3 * x - 5

def h(x):
    return 0.1 * 2 ** x

a = 1
b = 3

print("Antall rektangler | f(x)       | g(x)       | h(x) ")
print("------------------+------------+------------+------------")

for i in range(7):
    rektangler = 10 ** i
    fverdi = RektangelMetoden(f, a, b, rektangler)
    gverdi = RektangelMetoden(g, a, b, rektangler)
    hverdi = RektangelMetoden(h, a, b, rektangler)
    
    print("{:>17} | {:<10.7f} | {:<10.7f} | {:<10.7f} ".format(rektangler, fverdi, gverdi, hverdi))

Antall rektangler | f(x)       | g(x)       | h(x) 
------------------+------------+------------+------------
                1 | 2.0000000  | -4.0000000 | 0.4000000  
               10 | 7.8800000  | 1.4000000  | 0.8070029  
              100 | 8.5868000  | 1.9400000  | 0.8596309  
             1000 | 8.6586680  | 1.9940000  | 0.8650172  
            10000 | 8.6658667  | 1.9994000  | 0.8655570  
           100000 | 8.6665867  | 1.9999400  | 0.8656110  
          1000000 | 8.6666587  | 1.9999940  | 0.8656164  


De analytiske svaret for f(x) er $\frac{26}{3} \approx 8.666666667$, for g(x) er det $2$ og for h(x) er det $\approx 0.8656170245333$

Vi ser tydelig at desto flere rektangler vi har desto nærmere komme vi det analytiske svaret.  Likevel på grunn av at float tall kan bli upresiste med mange desimaler, kan det hende at hvis $\Delta x$ blir alt for liten kan den bli mindr epresis igjen.

### Variasjoner av metoden

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/RightRiemann2.svg/1280px-RightRiemann2.svg.png" style="width: 200px; float: left;">For å forbedre metoden kan man flytte på punktet der man måler høyden.  Firguren til venstre er et eksempel der man måler høyden på grafen på høyre siden av rektangelet i steden for på venstre siden.  Ved å kombinere ulike metoder kan mna få et enda mer presist resultat.  En kode med støtte for dette kan se slik ut

In [4]:
def RektangelMetoden(f, a, b, N = 100000, regel = "venstre"):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall rektangler
    # regel : Hvor man skal regne ut høyden ["venstre" (Standard) | "midten" | "hoyre" | float mellom 0 og 1]

    # Regne ut hva man må legge til i for å regne høyden på riktig sted
    i_regel = 0
    if type(regel) == float and 0 <= regel and regel <= 1:
        i_regel = regel
    elif regel.lower() == "midten":
        i_regel = 0.5
    elif regel.lower() == "hoyre":
        i_regel = 1
    elif regel.lower() != "venstre":
        # Kast en feil melding hvis regel argumentet er brukt feil
        raise Exception(f"Regel må være [\"venstre\" (Standard) | \"midten\" | \"hoyre\" | float mellom 0 og 1], men det ble spesifisert {regel}")

    # Bredden per rektangel
    dx = (b - a) / N
    # Arealet under grafen
    A = 0

    # Loop gjennom alle rekangelene
    for i in range(N):
        # Legg sammen høyden i rektanglene
        A += f(a + (i + i_regel) * dx)

    # Multipliser med bredden per integral for å finne arealet
    A *= dx

    # returner arealet under grafen f
    return A

Over kan vi se at om vi spesifiserer venstre får vi samme resultat som tidligere, men hvis vi spesifiserer midten måler vi på midten, eller på høyre side hvis vi spesiferer hoyre.  Under kan vi se resultatet av dette.

In [5]:
print("Venstre:", RektangelMetoden(f, 0, 1, regel = "venstre"))
print("Midten :", RektangelMetoden(f, 0, 1, regel = "midten"))
print("Hoyre  :", RektangelMetoden(f, 0, 1, regel = "hoyre"))

Venstre: 0.3333283333499996
Midten : 0.3333333333249928
Hoyre  : 0.3333383333499996


Her ser vi at resultatet er veldig likt, men dersom vi senker antall rektangler får vi ganske ulike svar.

In [6]:
print("Venstre:", RektangelMetoden(f, 0, 1, 5, "venstre"))
print("Midten :", RektangelMetoden(f, 0, 1, 5, "midten"))
print("Hoyre  :", RektangelMetoden(f, 0, 1, 5, "hoyre"))

Venstre: 0.24000000000000005
Midten : 0.33000000000000007
Hoyre  : 0.44000000000000006


Vi ser tydelig at å måle høyden på midten av søylen gir et mer nøyaktig svar enn å ta det på en hjørne.  Likevel med mange rektangler er det liten forskjell.

## Trapesmetoden

### Teori

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/TrapRiemann2.svg/1280px-TrapRiemann2.svg.png" style="width: 200px; float: left;">Trapesmetoden er veldig lik rektangelmetoden, men i steden for rektangler bruker den trapeser.  Vi kan tenke oss at man bruker rektangelmetoden og legger til en rettvinklet trekant øverst på hvert regktangel.  Da vil man få et mer presist resultat enn ved rektangelmetoden.  En illustrasjon av dette ser du til venstre.

For å uttrykke dette mattematisk får vi dette uttrykket:
$$\int_{a}^{b} f(x) dx \approx \lim_{N \rightarrow \infty} (\frac{\Delta x}{2} \sum_{i = 0}^{N-1} (f(a + i * \Delta x) + f(a + (i + 1) * \Delta x)))$$

Igjen er $\Delta x = \frac{b - a}{N}$

Vi har også faktorisert ut $\frac{\Delta x}{2}$ fra summne siden det er en faktor i alle leddene.

Uttrykket for arealet av et trapes er $A = \frac{(a + b) * h}{2}$.  Over er $a = f(a + i * \Delta x)$ og $b = f(a + (i + 1) * \Delta x)$ og $h = \Delta x$

### Kode

Før vi skal kode dette er det en del forandreinger vi bør gjøre på uttrykket for å få koden mer effektiv.  Med summen over må vi kalkulere hvert punkt to ganger.  Dette er ikke nødvendig og vi kan korte det ned med å kalkulere hvert punkt en gang og multiplisere svaret med 2.  Dette gjelder ikke punktene i x = a og x = b.  Så da må summnen utelukke sidde verdiene.  Da kan vi skrive det om slik:

$$\frac{\Delta x}{2} \sum_{i = 0}^{N-1} (f(a + i * \Delta x) + f(a + (i + 1) * \Delta x)) = \frac{\Delta x}{2} ( f(a) + f(b) + \sum_{i = 1}^{N - 2} 2 * f(a + i * \Delta x))$$

Videre for å gjøre få det til å bli enda raskere å kode kan vi faktorisere ut 2 siden dette er en felles faktor i alle leddene.  Dette gjelder heller ikke f(a) og f(b) så disse deler vi på 2.  Når vi først har faktorisert ut 2 får vi $\frac{\Delta x}{2} * 2$ som en felles faktor.  Her kan vi forkorte 2-erene mot hverandre.  Da får vi kun $\Delta x$ som felles faktor.  Hele uttrykket blir da seende slik ut:

$$\frac{\Delta x}{2} ( f(a) + f(b) + \sum_{i = 1}^{N - 2} 2 * f(a + i * \Delta x)) = \Delta x * (\frac{f(a) + f(b)}{2} + \sum_{i = 1}^{N - 2} f(a + i * \Delta x))$$

Det uttrykket vi da får vil gi oss det mest effektive programmet.

In [7]:
def TrapesMetoden(f, a, b, N = 100000):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall trapeser

    # Bredden per trapes
    dx = (b - a) / N
    # Arealet under grafen
    A = (f(a) + f(b)) / 2

    # Loop gjennom alle trapesene
    for i in range(1, N):
        # Legg sammen høyden i rektanglene
        A += f(a + i * dx)

    # Multipliser med bredden
    A *= dx

    # returner arealet under grafen f
    return A

In [8]:
Areal = TrapesMetoden(f, 1, 2)
print(Areal)

2.333333333350027


Med denne enkle testen som vi også brukte på rektangelmetoden ser vi at Trapesmetoden er mer presis enn rektangelmetoden.  Og her kommer vi nærmere $\frac{7}{3}$ enn vi gjorde i stad.

### Ulikt antall trapeser

In [9]:
print("Antall trapeser   | f(x)       | g(x)       | h(x) ")
print("------------------+------------+------------+------------")

for i in range(7):
    trapeser = 10 ** i
    fverdi = TrapesMetoden(f, a, b, trapeser)
    gverdi = TrapesMetoden(g, a, b, trapeser)
    hverdi = TrapesMetoden(h, a, b, trapeser)
    
    print("{:>17} | {:<10.7f} | {:<10.7f} | {:<10.7f} ".format(trapeser, fverdi, gverdi, hverdi))

Antall trapeser   | f(x)       | g(x)       | h(x) 
------------------+------------+------------+------------
                1 | 10.0000000 | 2.0000000  | 1.0000000  
               10 | 8.6800000  | 2.0000000  | 0.8670029  
              100 | 8.6668000  | 2.0000000  | 0.8656309  
             1000 | 8.6666680  | 2.0000000  | 0.8656172  
            10000 | 8.6666667  | 2.0000000  | 0.8656170  
           100000 | 8.6666667  | 2.0000000  | 0.8656170  
          1000000 | 8.6666667  | 2.0000000  | 0.8656170  


Over ser vi samme kode som vi testet rektangelmetoden med.  Ved å sammenlikne disse tabellene, ser vi at trapesmetoden finner riktig svar med færre trapeser enn rektangelmetoden trengte rektangler.

## Simpsons metode

### Teori

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Simpsons_method_illustration.svg/1024px-Simpsons_method_illustration.svg.png" style="width: 200px; float: left">

Simpsonsmetode prøver igjen å være enda mer presis enn trapesmetoden.  Trapesmetoden finner en rett linje gjennom to punkter på grafen som er veldig nærme hverandre.  Simpsons metode finner et andregradsuttrykk $P(x)$ som går gjennom 3 punkter veldig nærme hverandre på grafen $f(x)$.  Disse punktene bruker vi til å regne ut $P(x)$, og da har vi toppen av en smal figur i et lite intervall.

Vi kan kalle punktene $a$, $m$ og $b$.  $m$ ligger midt i mellom $a$ og $b$.  Avstanden mellom $a$ og $b$ kan vi kalle $2 * h$.  På figuren til venstre ser vi en illustrasjon av dette.

Siden vi vet at $P(x)$ er et andregradsuttrykk kan vi skrive det på en generell form.

$$P(x) = A * x^2 + B * x + C$$

Da ser vi at vi må finne et uttrykk for $A$, $B$ og $C$ som gjør at $P(x)$ går gjennom alle punktene.

For å gjøre det enklere kan vi si at $m = 0$, da vil $a = -h$ og $b = h$

Da vet vi at $C = f(m)$, siden $P(m) = P(0) = C = f(m)$

Vi kan videre si dette:

$$f(a) = P(a) = A * h^2 - B * h + f(m)$$
$$f(b) = P(b) = A * h^2 + B * h + f(m)$$

Setter vi da inn det første uttrykket i det andre får vi:

$$A = \frac{f(b) - B * h - f(m)}{h^2}$$

Setter vi videre det andre uttrykket inn i det første gitt at det over stemmer finner vi at $B$ er dette:

$$B = \frac{f(b) - f(a)}{2 * h}$$

Setter vi videre inn $B$ i uttrykket med $A$, får vi dette:

$$A = \frac{f(a) - 2 * f(m) + f(b)}{2 * h^2}$$

Uttrykket $P(x)$ blir da slik:

$$P(x) = \frac{f(a) - 2 * f(m) + f(b)}{2 * h^2}x^2 + \frac{f(b) - f(a)}{2}x + f(m)$$

Da har vi et uttrykk for $P(x)$ og dette kan vi analytisk integrere for å finne arealet under $P(x)$ i intervallet $[a, b]$.  Dette gjør vi analytisk slik.

$$\int_{-h}^{h} P(x) dx = \int_{-h}^{h} (A x^2 + B x + C) dx  = [\frac{1}{3} A x^3 + \frac{1}{2} B x^2 + c x]_{-h}^{h} = \frac{1}{3} A h^3 + \frac{1}{2} B h^2 + c h + \frac{1}{3} A h^3 - \frac{1}{2} B h^2 + c h = \frac{2}{3} A h^3 + 2 C h$$

Så setter vi inn $A$, $B$ og $C$ og får dette:

$$\frac{2}{3} * \frac{f(a) - 2 * f(m) + f(b)}{2 * h^2} * h^3 + 2 f(m) h = \frac{h}{3} * (f(a) + 4f(m) + f(b))$$

Da har vi et uttrykk for å finne arealet under en smal figur.  For å få hele det bestemte integralet presist må vi som med rektangel og trapesmetoden legge sammen mange små skiver av denne figuren.  Da får vi denne summen.

$$\int_{a}^{b} f(x) dx \approx \lim_{N \rightarrow \infty} (\frac{h}{3} \sum_{i = 0}^{N-1} (f(a + i * 2 * h) + 4f(a + (i * 2 + 1) * h) + f(a + (i * 2 + 2) * h)))$$

Der:

$$2h = \frac{b - a}{N}$$

### Kode

Dette uttrykket er greit å lese, men bør endres litt for å få et effektivt program.  Da kan vi først se at også her er det mange verdier der vi regner ut $f(x)$ av samme verdi to ganger.  Dette gjelder igjen ikke $f(a)$ og $f(b)$.  Så derfor kan vi korte ned summen i begge ender.  Da må vi også huske å legge til leddet i midten.  Da får vi et uttrykk som dette:

$$\frac{h}{3} \sum_{i = 0}^{N-1} (f(a + i * 2 * h) + 4f(a + (i * 2 + 1) * h) + f(a + (i * 2 + 2) * h)) = \frac{h}{3} * (f(a) + 4f(a + h) + f(b) + \sum_{i = 1}^{N - 2} (2f(a + i * 2 * h) + 4f(a + (i * 2 + 1) * h)))$$

Her kunne vi også hatt $4f(b - h)$ uten for summen, og inni summen hatt $4f(a + (i * 2 - 1) * h)$ isteden for $4f(a + (i * 2 + 1) * h)$, men her er det hipp som happ hva man velger.

In [10]:
def SimpsonsMetode(f, a, b, N = 100000):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall søyler

    # Bredden per søyle
    h = (b - a) / (2 * N)
    # Arealet under grafen
    A = f(a) + 4 * f(a + h) + f(b)

    for i in range(1, N):
        A += 2 * f(a + i * 2 * h) + 4 * f(a + (i * 2 + 1) * h)

    # Multipliser med h / 3
    A *= h / 3

    # returner arealet under grafen f
    return A

In [11]:
Areal = SimpsonsMetode(f, 1, 2)
print(Areal)

2.333333333333348


Denne enkle testen vi har brukt på de tidligere metodene, viser igjen at Simpsonsmetode er enda mer presis enn både Trapesmetoden og rektangelmetoden.  Dette er enda nærmere $\frac{7}{3}$ som er det analytiske svaret.

In [12]:
Areal = SimpsonsMetode(f, 1, 2, 1)
print(Areal)

2.333333333333333


Her er det verdt å merke seg at siden $f(x)$ er et andregradsuttrykk, vil simpsons metode finne dette uttrykket og danne den formen for hvert av søylene den kalkulerer arealet til.  Derfor holder det med en søyle for å finne integralet til et andregradsuttrykk.

### Ulikt antall figurer

In [13]:
print("Antall figurer    | f(x)              | g(x)              | h(x) ")
print("------------------+-------------------+-------------------+-------------------")

for i in range(7):
    figurer = 10 ** i
    fverdi = SimpsonsMetode(f, a, b, figurer)
    gverdi = SimpsonsMetode(g, a, b, figurer)
    hverdi = SimpsonsMetode(h, a, b, figurer)
    
    print("{:>17} | {:<10.15f} | {:<10.15f} | {:<10.15f} ".format(figurer, fverdi, gverdi, hverdi))

Antall figurer    | f(x)              | g(x)              | h(x) 
------------------+-------------------+-------------------+-------------------
                1 | 8.666666666666666 | 2.000000000000000 | 0.866666666666667 
               10 | 8.666666666666666 | 2.000000000000000 | 0.865617135478134 
              100 | 8.666666666666668 | 2.000000000000000 | 0.865617024544479 
             1000 | 8.666666666666664 | 2.000000000000000 | 0.865617024533379 
            10000 | 8.666666666666673 | 2.000000000000003 | 0.865617024533377 
           100000 | 8.666666666666655 | 1.999999999999999 | 0.865617024533381 
          1000000 | 8.666666666666552 | 1.999999999999999 | 0.865617024533364 


Over er igjen den kjente tabellen.  Simpsons metode er merkbart treigere enn både trapesmetoden og rektangelmetoden.  Likevel kommer den til et nøyaktigsvar med enda færre søyler.  Da kan man likevel kutte ned på antallsøyler så blir den rask nok.  Vi kan også se at med f(x) og g(x) så minker nøyaktigheten når antall figurer øker.  Dette kommer av unøyaktigheten av lagring som floats.

## Monte Carlo

### Teori

Det er flere metodeer som bruker monteCarlo metoden for å integrere.  I denne Notebooken bruker vi en metode som går ut på å finne finne høyden på fuksjonen på mange tilfeldige punkter.  Deretter finner man den gjennomsnittlige høyden og multipliserer med lengden på intervallet man skal integrere.  Dersom man finner mange nok tilfeldige punkter og algoritmen for tilfeldighet er tilfeldig vil dette gå mot integralet av funksjonen.

Et matematisk uttrykk vil da bli seende slik ut

$$\int_a^b {f(x) dx} \approx \frac{b - a}{N} * \sum_{i = 1}^{N-1} {f(x_i)}$$

Her er $x_i$ et tilfeldig punkt i integravet $[a , b]$.

### Kode

In [14]:
import random

def MonteCarlo(f, a, b, N = 100000, randomAlgorithm = random.random):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall punkter
    # randomAlgorithm : En algoritme som gir tilfeldige tall

    # Lengden på intervallet
    w = (b - a)

    # En variabel for å lagre summen av høyden
    sum = 0

    # En loop for kjører N ganger
    for i in range(N):
        # Regn ut en tilfeldig x verdi i intervallet
        x = randomAlgorithm() * w + a
        # Legg til funksjonsverdien til f i punktet x til summen
        sum += f(x)

    # Finn den gjnnomsnittlige høyden og multipliser den med bredden av integralet
    return sum / N * w

In [15]:
Areal = MonteCarlo(f, 1, 2)
print(Areal)

2.3346993586263114


Over ser vi at metoden funker greit, men resultet vil endre seg gang vi kjører den, siden mye av metoden er bygget på tilfeldigheter.  I dette tilfellet traff den på 2 desimaler, det er ikke særlig nøyaktig.

### Ulikt antall punkter

In [16]:
print("Antall punkter    | f(x)       | g(x)       | h(x) ")
print("------------------+------------+------------+------------")

for i in range(7):
    punkter = 10 ** i
    fverdi = MonteCarlo(f, a, b, punkter)
    gverdi = MonteCarlo(g, a, b, punkter)
    hverdi = MonteCarlo(h, a, b, punkter)
    
    print("{:>17} | {:<10.7f} | {:<10.7f} | {:<10.7f} ".format(punkter, fverdi, gverdi, hverdi))

Antall punkter    | f(x)       | g(x)       | h(x) 
------------------+------------+------------+------------
                1 | 3.4931367  | 4.0601704  | 1.1052216  
               10 | 7.4938015  | 0.6953055  | 1.0305224  
              100 | 9.0483322  | 2.0224415  | 0.8668867  
             1000 | 8.8712598  | 2.0986417  | 0.8460583  
            10000 | 8.6742409  | 2.0071613  | 0.8677591  
           100000 | 8.6670804  | 1.9920631  | 0.8662391  
          1000000 | 8.6686969  | 1.9982573  | 0.8657312  


Vi merker at også Monte Carlo er ganske treig, men ikke like treig som Simpsons metode.  Vi ser også at antall punkter øker nøyaktigheten.  Til tross for mange punkter så er den ikke særlig nøyaktig heller.

## Guss Kvadratur

### Teori

Gauss kvadratur integrasjon går ut på å regne ut integralet til en funksjon $f$ ved å regne ut noen verdier på grafen og multiplisere med med noen vekter.  Summen av dette vil da være tilnærmet integralet til $f$.  Man kan tenke seg at Guass kvadratur finner et polynom $P$ som går gjennom alle disse punktene og videre regnet ut arealet under polynomet $P$.  Graden til $P$ vil da bestemme nøyaktigheten til metoden, og den vil gi eksakte svar for polynomer opp til graden av $P$, gitt at man bruker metoden analytisk.

#### Framgangsmåte

Som sagt så går metoden ut på å finne summen av fuksjonsverdien i visse punkter multiplisert med en vekt.  Gauss kvadratur tar utganspunktet i intervallet $[-1, 1]$, og dette gjør også bevisene lettere.  Videre skal vi utvide det til alle intervaller.  Mattematisk blir dette uttrykt slik:

$$\int_{-1}^{1}{f(x) dx} \approx \sum_{i = 0}^{N - 1}{w_i f(x_i)}$$

Der $N = \frac{n + 1}{2}$, der $n$ er graden til $P$.  Årsaken til denne likningen står nedenfor.

Et kode eksempel på denne fuksjonen ser vi under.

In [17]:
def regnSUM(f, punkter, vekter):
    # f : En fuksjon å bruke
    # punkter : puntker å regne ut funksjonsverdien til
    # vekter : vektene vi må multiplisere hver fuksjonsverdi med

    ret = 0

    # Regn ut summan av f(x) til alle punktene, multiplisert med den tilsvarende vekten til punktet
    for i in range(len(punkter)):
        ret += vekter[i] * f(punkter[i])

    return ret

La oss sette $N = 2$.  Da natar vi at uttrykket under stemmer:

$$\int_{-1}^1{f(x) dx} \approx w_0 f(x_0) + w_1 f(x_1)$$

Siden vi har fire ukjente $w_0$, $x_0$, $w_1$ og $x_1$, trenger vi å sette opp fire likninger for å finne ut hva de ukjente er.
Dette kan vi gjøre ved å se på integralet til disse 4 enkle polynomene og bruke antagelsen vår til å løse disse verdiene.  Dette gjør vi ved å sette integralet mellom -1 og 1 til polynomene lik $w_0 f(x_0) + w_1 f(x_1)$, der f er polynomene.

<u>Polynom 1: $x^0$</u>

$$\int_{-1}^1{x^0 dx} = 2 = w_0 f(x_0) + w_1 f(x_1)$$

$$2 = w_0 + w_1$$

<u>Polynom 2: $x^1$</u>

$$\int_{-1}^1{x^1 dx} = 0 = w_0 f(x_0) + w_1 f(x_1)$$

$$0 = w_0 x_0 + w_1 x_1$$

<u>Polynom 3: $x^2$</u>

$$\int_{-1}^1{x^2 dx} = \frac{2}{3} = w_0 f(x_0) + w_1 f(x_1)$$

$$\frac{2}{3} = w_0 x_0^2 + w_1 x_1^2$$

<u>Polynom 4: $x^3$</u>

$$\int_{-1}^1{x^3 dx} = 0 = w_0 f(x_0) + w_1 f(x_1)$$

$$0 = w_0 x_0^3 + w_1 x_1^3$$

Løser vi disse likningene får vi dette svaret:

$w_0 = 1$

$w_1 = 1$

$x_0 = -\frac{\sqrt{3}}{3}$

$x_1 = \frac{\sqrt{3}}{3}$

Vi får egentlig to løsninger, men vi trenger bare regne med en av dem.  Hvis vi ser på den andre løsningen så ser vi at $w_0$ og $w_1$ er like, mens $x_0 = \frac{\sqrt{3}}{3}$ og $x_1 = -\frac{\sqrt{3}}{3}$.  Men det er enklest å tenke oss at $x_0 < x_1$, selv om det ikke har noe å si.

Nå som vi har løst de fire ukjente variablene kan vi regne ut integralet til $f$ i intervallet $[-1, 1]$.  Det blir slik:

$$\int_{-1}^1{f(x) dx} \approx f(-\frac{\sqrt{3}}{3}) + f(\frac{\sqrt{3}}{3})$$

Dette svaret er eksakt hvis $f$ er et polynom med graden 3 eller lavere, ellers er det en tilnærming.

Vi kan utvide metoden ved å la $N$ være et høyere tall.  Da får vi flere ukjente og for å løse likningen må vi ha flere likninger.  Dersom $N$ øker med 1, vil vi få to nye ukjente.  Da trenger vi to nye likninger, og graden til $P$ øker da med to.  Vi kan uttrykke dette slik $n = 2N - 1$.  Vi må trekke fra 1, fordi hvis vi vil at $P$ skal ha graden 1, trenger vi 2 likninger, og med 2 likninger trenger vi 2 ukjente.  Og vi får 2 ukjente hvis $N = 1$.

Denne likningen kan vi skrive om til dette $N = \frac{n + 1}{2}$, som vi brukte over.  Det kan være verdt å merke seg at $n$ må være et positivt oddetall, siden $N$ må være et positivt heltall.

#### Legendre Polynomer

Det er litt tungvint å finne alle disse likningene og så løse dem.  Så derfor bruker man legendeprolynomer og finner nullpunktene til disse polynomene.  Disse nullpunktene ligger akkurat på $x_i$ svarene vi hadde fått dersom vi brukte likningene over til å finne $x_i$.  For å finne vektene kan vi igjen bruke disse polynomene.

En av de rekursive formlene for polynomene ser slik ut:

$$(n + 1) P_{n + 1}(x) = (2n + 1)x P_n(x) - n P_{n - 1}(x)$$

Den har initsialbetingelsene $P_0(x) = 1$ og $P_1(x) = x$.  Her er $n$ graden til Legende Polynomet.  Siden hvert av legende polynomene har like mange nullpunkter som graden, vil $n$ representere antall nullpunkter, og da også antall $x_i$ verdier.

Sånn den er nå er den litt tungvint å kode, så la oss først bytte ut $n + 1$ med $n$.  Da blir formelen seende slik ut:

$$n P_n(x) = (2n - 1) x P_{n - 1}(x) - n P_{n - 2}(x)$$

Videre kan vi få $P_n(x)$ alene på en side ved å dele på $n$

$$P_n(x) = \frac{(2n - 1) x P_{n - 1}(x) - n P_{n - 2}(x)}{n}$$

Videre for å kode dette trenger vi en måte å lagre polynomer.  Da bruker vi numpy og der lagrer man polynomer som lister, se <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.polynomial.polynomial.Polynomial.html#numpy.polynomial.polynomial.Polynomial">her</a>.

Da har vi alt vi trenger for å regne ut Lengendre Polynomene

In [18]:
# Først må vi importere polynom delen fra numpy, eventuelt pylab
from numpy.polynomial import polynomial as P

# Denne funksjonene returnerer Legendre polynomene
def legendrePolynom(n):
    # n : graden til Legendre polynomet

    # Dette er initsialbetingelsene til den rekursive formelen
    if n == 0:
        return [1]

    if n == 1:
        return [0, 1]

    # Så bruker vi den rekursive formelen videre
    # n P_n(x) = (2n - 1)xP_{n - 1}(x) - (n - 1) P_{n - 2}(x)

    ret = P.polymul([0, 2 * n - 1], legendrePolynom(n - 1))
    ret = P.polyadd(ret, P.polymul(legendrePolynom(n - 2), [1 - n]))
    ret = P.polymul(ret, [1 / n])

    # Returner resultatet
    return ret


Videre for å regne ut vektene til hvert av polynomene kan vi bruke denne formelen.

$$w_i = \frac{2}{(1 - x_i^2) (P'_n(x_i))^2}$$

Å kode denne er ganske rett fram, den blir som vist under.  Her tar vi den deriverte til Legende polynomet som argument for å effektivisere litt.

In [19]:
def vekt(polyDer, x):
    # polyDer : Den deriverte til et legendre polynom
    # x : x å finne vekten til

    # w_i = 2 / ((1 - x_i^2) * (P_n'(x_i))^2 )

    return 2 / ((1 - x ** 2)  * (P.polyval(x, polyDer) ** 2))

#### Bevis

Da har vi komt såpass langt at metoden nesten er ferdig.  Siden alt hittil kan gjøre analytisk med små verdier for $n$, kan vi bevise at denne metoden funker for polynomer med en grad mindre eller lik $n$.

Da kan vi tenke oss et polynom

$$f(x) = a x^3 + b x ^2 + c x + d$$

Vi kan finne det bestemte integralet mellom -1 og 1 slik:

$$\int_{-1}^1{f(x) dx} = [\frac{a}{4}x^4 + \frac{b}{3}x^3 + \frac{c}{2}x^2 + d x]_{-1}^1 = \frac{2 b}{3} + 2 d$$

Ved å bruke Gauss Kvadratur gjør vi det slik

$$\int_{-1}^1{f(x) dx} = f(-\frac{\sqrt{3}}{3}) + f(\frac{\sqrt{3}}{3}) = \frac{2 b}{3} + 2 d$$

Dette beviser at dersom $n$ er 3 så kan vi finne det bestemte integralet mellom -1 og 1 eksakt dersom fuksjonene et et tredjegradspolynom.  Dette beviser ikke at den er eksakt for større polynomer, men gjør man bevisene ser man at det stemmer.

#### Andre intervaller

Hittil har vi bare funnet det bestemte integralet mellom -1 og 1, og det er kun dette intervallet integrasjonen funker.  Likevel kan vi brukte substitusjon, og bytte ut $x$.  La oss si at vi setter inn $u$.  Da "skviser" vi intervaller $[a, b]$ inn i $[-1, 1]$.

$$x(u) = \frac{b - a}{2} u + \frac{a + b}{2}$$

Over er uttrykket vi skal bytte ut x med.  Hvis vi nå setter $x(-1)$ får vi $a$ og setter vi $x(1)$ så får vi $b$.  Siden $u(x)$ er linjær, vil en verdi mellom $a$ og $b$ også være mellom $-1$ og $1$ med samme forhold. Videre bruker vi substitusjon.

$$x'(u) = \frac{dx}{du}$$

$$\int_a^b {f(x) dx} = \int_{-1}^1 {f(\frac{b - a}{2} u + \frac{a + b}{2}) x'(u) du}$$

$$x'(u) = \frac{b - a}{2}$$

$x'(u)$ er bare en konstant, så den kan vi sette untenfor integralet.

$$\int_a^b {f(x) dx} = \frac{b - a}{2} \int_{-1}^1 {f(\frac{b - a}{2} u + \frac{a + b}{2}) du}$$

Dette er ikke så vanskelig å kode. Vi ser bort ifra integralet for nå.  Funksjonen under returnerer en ny fuksjon der som vi kan integrere mellom -1 og 1, i stedet for mellom $a$ og $b$.

In [20]:
# En fuksjon som returerer en annen funksjon for å skise integralet mellom a og b til integralet mellom -1 og 1
def endreIntervall(f, a, b):
    # f : en funksjon
    # a : starten på et intervall
    # b : slutten på et intervall

    # En funksjon å returnere
    def ret(x):
        # x : en verdi å regne ut den tilsvarende verdien til

        return (b - a) / 2 * f((b - a) / 2 * x + (a + b) / 2)

    # Returner funksjonen
    return ret

### Kode

Da har vi alt vi trenger for å finne bestemte integraler ved Gauss kvadratur.  Koden er under.

In [21]:
# En funksjon som bruker Gauss kvadrartur til å regne ut det bestemte integralet til en funksjon
def GaussKvadratur(f, a = -1, b = 1, N = 7):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Hvilken grad vi skal ha av polynomet

    # Sjekk at N er et oddetall større enn 0, årsaken er at vi må ha et partall antall ukjente i likningen
    # For eksemper hvis N = 1, er integralet det samme som w_1 * f(x_1) + w_2 * f(x_2), altså 4 ukjente
    if N % 2 == 0 or N < 1:
        # Kast en feil hvis N er spesiefisert feil
        raise ValueError("N må være et oddetall større enn 0")

    # Regn ut legendre graden til den tilsvarende graden av polynomet som ble spesifisert
    n = int((N + 1) / 2)

    # Skvis funksjonen ned så a blir liggende på -1 og b på 1.
    nyF = endreIntervall(f, a, b)

    # Regn ut Legendre polynomet med graden n
    lPoly= legendrePolynom(n)

    # Regn ut nullpunktene til Legendre polynomet.  Dette blir x verdiene vi skal sjekke
    nullpunkter = P.polyroots(lPoly)

    # Regn ut den deriverte til Legendre polynomet.  Brukes til å regne ut vektene til hver x verdi
    polyDer = P.polyder(lPoly)

    vekter = []

    # Lag en liste med de tilsvarende vektene til punktene vi fant
    for i in nullpunkter:
        vekter.append(vekt(polyDer, i))

    # regn ut summen vi nå har
    SUM = regnSUM(nyF, nullpunkter, vekter)

    # Returner summen vi fant
    return SUM

In [22]:
Areal = GaussKvadratur(f, 1, 2)
print(Areal)

2.3333333333333326


Over ser vi resultatet med test funksjonen vi hadde, og metoden treffer veldig bra.  Vi bommer litt, men dette har med avrundninger når vi regner med numeriske verdier.

### Ulik grad til polynomet $P(x)$

In [23]:
print("Graden til polynomet | f(x)                | g(x)                | h(x) ")
print("---------------------+---------------------+---------------------+---------------------")

for i in range(17):
    grad = 2 * i + 1
    fverdi = GaussKvadratur(f, a, b, grad)
    gverdi = GaussKvadratur(g, a, b, grad)
    hverdi = GaussKvadratur(h, a, b, grad)
    
    print("{:>20} | {:<10.17f} | {:<10.17f} | {:<10.17f} ".format(grad, fverdi, gverdi, hverdi))

Graden til polynomet | f(x)                | g(x)                | h(x) 
---------------------+---------------------+---------------------+---------------------
                   1 | 8.00000000000000000 | 2.00000000000000000 | 0.80000000000000004 
                   3 | 8.66666666666666607 | 1.99999999999999911 | 0.86491992374991955 
                   5 | 8.66666666666666430 | 1.99999999999999956 | 0.86561416626771326 
                   7 | 8.66666666666666252 | 1.99999999999999556 | 0.86561701832296523 
                   9 | 8.66666666666665364 | 1.99999999999999134 | 0.86561702452502098 
                  11 | 8.66666666666666963 | 1.99999999999999556 | 0.86561702453337097 
                  13 | 8.66666666666668561 | 2.00000000000000311 | 0.86561702453338008 
                  15 | 8.66666666666668739 | 1.99999999999999756 | 0.86561702453338030 
                  17 | 8.66666666666671226 | 2.00000000000001954 | 0.86561702453338196 
                  19 | 8.66666666666672647 | 2.

Vi merker at metoden er nokså rask, og den gir veldig nøyaktige resultater med lave grader av polynomet.  For å se hvor nøyaktig metoden er enklere kan vi se på differansen til det analytiske svaret.

In [24]:
import math

print("Graden til polynomet | f(x)                | g(x)                | h(x) ")
print("---------------------+---------------------+---------------------+---------------------")

for i in range(17):
    grad = 2 * i + 1
    fverdi = abs(26 / 3 - GaussKvadratur(f, a, b, grad))
    gverdi = abs(2 - GaussKvadratur(g, a, b, grad))
    hverdi = abs(3 / (5 * math.log(2)) - GaussKvadratur(h, a, b, grad))
    
    print("{:>20} | {:<10.17f} | {:<10.17f} | {:<10.17f} ".format(grad, fverdi, gverdi, hverdi))

Graden til polynomet | f(x)                | g(x)                | h(x) 
---------------------+---------------------+---------------------+---------------------
                   1 | 0.66666666666666607 | 0.00000000000000000 | 0.06561702453337803 
                   3 | 0.00000000000000000 | 0.00000000000000089 | 0.00069710078345853 
                   5 | 0.00000000000000178 | 0.00000000000000044 | 0.00000285826566482 
                   7 | 0.00000000000000355 | 0.00000000000000444 | 0.00000000621041285 
                   9 | 0.00000000000001243 | 0.00000000000000866 | 0.00000000000835709 
                  11 | 0.00000000000000355 | 0.00000000000000444 | 0.00000000000000711 
                  13 | 0.00000000000001954 | 0.00000000000000311 | 0.00000000000000200 
                  15 | 0.00000000000002132 | 0.00000000000000244 | 0.00000000000000222 
                  17 | 0.00000000000004619 | 0.00000000000001954 | 0.00000000000000389 
                  19 | 0.00000000000006040 | 0.

Her ser vi at nøaktigheten øker for høyere grad av $P(x)$, men at for høy grad av $P(x)$ også gir upresise svar.

### Forbedringer

En mulighet for å gjøre denne metoden veldig rask er hvis vi hardcoder inn verdiene for $x_i$ og $w_i$ eller lagrer dem i en slags cache.  Dette funker dårlig til å vise fram metoden, men dersom vi gjør dette blir metoden utrolig rask og  presis.

## Drøfting

Vi ser at de tre første metoden er nokså like.  Alle deler opp integralet i små figurer.  Rektangelmetoden bruekr raktangler som en figur.  Dette funker veldig bra for å forklare en enkel måte å integrere analytisk med.  Den er også rask og nokså nøyaktig.

Trapesmetoden er nesten like rask som rektangelmetoden og den er mye mer nøyaktig.  Så for å få nøyaktige svar fort funker nok denne bedre enn rektangelmetoden i alle sammenhenger.  Unntaket er kanskje hvis man skal forklare en numerisk integrasjonsmetode.

Simpsonsmetode tar det hele et skritt videre.  Den er mye mer nøyaktig enn trapesmetoden, men den bruker også mye lenger tid.  Så denne funker veldig bra hvis nøyaktighet er viktigere enn hvor rask metoden er.

Monte Carlo integrasjon er verken nøyaktig eller rask.  Den funker bra til å påpeke et prinsipp, men den funker dårlig til så å si alle praktiske forhold.

Gauss Kvadratur er mye mer nøyaktig enn alle de andre metodene.  Man kan også forbedre koden over for å få den ufattelig rask også.  Denne funker supert for å integrere rask og nøyaktig.